In [29]:
import warnings

import numpy as np
import pandas as pd
from google.cloud import storage

from scratch.compare_msi_output_utils import (
    get_terra_outputs,
    combine_dis_wavg,
    save_msi_outputs,
)

pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 50)
pd.set_option("display.max_info_columns", 30)
pd.set_option("display.max_info_rows", 20)
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_seq_items", 20)
pd.set_option("display.width", 200)
pd.set_option("expand_frame_repr", True)
pd.set_option("mode.chained_assignment", "warn")

In [30]:
old_samples = get_terra_outputs(
    namespace="broad-firecloud-ccle", workspace="DepMap_WGS_CN"
)
new_samples = get_terra_outputs(
    namespace="broad-firecloud-ccle", workspace="depmap_omics_wgs"
)

sample_ids = set(old_samples["sample_id"]).intersection(set(new_samples["sample_id"]))
old_samples = old_samples.loc[old_samples["sample_id"].isin(sample_ids)]
new_samples = new_samples.loc[new_samples["sample_id"].isin(sample_ids)]

In [31]:
storage_client = storage.Client(project="depmap-omics")
old_samples = save_msi_outputs(storage_client, old_samples, old_or_new="old")
new_samples = save_msi_outputs(storage_client, new_samples, old_or_new="new")

# Scores

Comparing old and new msisensor2 scores (percentage of somatic sites) per sample

In [32]:
score_comp = old_samples[["sample_id", "msisensor2_score"]].merge(
    new_samples[["sample_id", "msisensor2_score"]],
    on="sample_id",
    how="inner",
    suffixes=("_old", "_new"),
)

score_comp

,sample_id,msisensor2_score_old,msisensor2_score_new
0,CDS-1tmwd7,90.86,90.57
1,CDS-00Nrci,3.68,3.56
2,CDS-0sdrSB,90.46,90.19
3,CDS-0bV15m,2.19,2.16
4,CDS-0e3PRe,2.39,2.41
5,CDS-0b4jFH,4.76,4.83
6,CDS-BEbnKg,82.96,83.09
7,CDS-xvtYY4,46.85,46.41
8,CDS-0JJKl3,3.19,3.19
9,CDS-0q5dcO,2.00,2.00


In [33]:
score_mse = np.square(
    score_comp["msisensor2_score_new"] - score_comp["msisensor2_score_old"]
).mean()
score_med_abs_dev = np.abs(
    score_comp["msisensor2_score_new"] - score_comp["msisensor2_score_old"]
).median()
score_max_abs_dev = np.abs(
    score_comp["msisensor2_score_new"] - score_comp["msisensor2_score_old"]
).max()

print(f"Mean squared error: {score_mse}")
print(f"Median absolute deviation: {round(score_med_abs_dev, 2)}")
print(f"Max absolute deviation: {round(score_max_abs_dev, 2)}")

Mean squared error: 0.02525000000000052
Median absolute deviation: 0.06
Max absolute deviation: 0.44


# Read count distribution

Comparing old and new msisensor2 read count distributions

## Per sample per site

In [34]:
with warnings.catch_warnings(action="ignore"):
    old_repeats = combine_dis_wavg(old_samples)
    new_repeats = combine_dis_wavg(new_samples)

repeats_comp = old_repeats.join(
    new_repeats, how="outer", lsuffix="_old", rsuffix="_new"
)

# make sure the outer join was effectively an inner one (i.e. indexes match)
assert len(repeats_comp) == len(old_repeats) == len(new_repeats)

In [35]:
repeats_comp

CDS-1tmwd7_old  CDS-00Nrci_old  CDS-0sdrSB_old  CDS-0bV15m_old  CDS-0e3PRe_old  CDS-0b4jFH_old  CDS-BEbnKg_old  CDS-xvtYY4_old  CDS-0JJKl3_old  \
chr  loc       left_flank repeat right_flank                                                                                                                                                   
chr1 100077146 AAATC      15[T]  CAGAC             11.000000       14.714286       10.687500       15.000000       15.034483            15.0       12.210526       14.000000       15.030303   
     100218786 AAAAA      12[T]  ACTAA             10.000000       12.000000       11.333333       12.000000       11.956522            12.0       11.000000       10.952381       12.000000   
     10072379  ATCGC      10[T]  ATTGC              8.413793        9.892857        9.352941       10.000000       10.000000            10.0       10.032258        9.972973       10.000000   
     100990636 CTATT      10[A]  GATAC              9.000000       10.000000       10.434783       10.000000       10.000000            10.0        9.388889        9.892857       10.000000   
     103107961 ATTGC      14[T]  CTTGA             12.103448       13.813953        9.965517       14.000000       13.916667            14.0       10.777778       13.000000       14.000000   
...                                                      ...             ...             ...             ...             ...             ...             ...             ...             ...   
chr9 94301443  AATGG      11[A]  TGTTG              9.555556       10.902439        8.757143       10.972973       11.000000            11.0       10.060606       10.973684       11.000000   
     94664048  GGAAT      13[A]  GCCAG             10.939394       12.857143        9.603175       13.000000       13.000000            13.0        9.518519       12.529412       13.000000   
     95012920  TTTTG      12[T]  AACGT             11.210526       12.100000       10.000000       12.000000       12.000000            12.0       10.160000       11.315789       12.000000   
     96645797  TCTCG      14[A]  GGAAA             12.136364       13.952381       11.690476       14.000000       14.000000            14.0       12.666667       13.692308       14.000000   
     97439221  TTTAC      13[T]  AATAT             11.500000       12.928571       11.200000       13.064516       13.000000            13.0       10.928571       12.368421       12.948718   

                                              CDS-0q5dcO_old  CDS-fSBbM3_old  CDS-099jzP_old  CDS-0A4mDu_old  CDS-01m27W_old  CDS-051xn7_old  ...  CDS-0q5dcO_new  CDS-0e3PRe_new  CDS-fSBbM3_new  \
chr  loc       left_flank repeat right_flank                                                                                                  ...                                                   
chr1 100077146 AAATC      15[T]  CAGAC             15.000000       15.000000       15.000000       15.000000       15.000000       15.000000  ...       15.000000       15.034483       15.000000   
     100218786 AAAAA      12[T]  ACTAA             12.032258       12.000000       11.894737       11.970588       12.000000       12.000000  ...       12.032258       11.956522       12.000000   
     10072379  ATCGC      10[T]  ATTGC             10.000000       10.000000       10.045455        9.976744        9.961538        9.972222  ...       10.000000       10.000000       10.000000   
     100990636 CTATT      10[A]  GATAC             10.030303       10.000000       10.000000       10.000000       10.000000       10.000000  ...       10.030303       10.000000       10.000000   
     103107961 ATTGC      14[T]  CTTGA             14.000000       14.000000       14.000000       13.916667       14.000000       14.000000  ...       14.000000       13.916667       14.000000   
...                                                      ...             ...             ...             ...             ...             ...  ...             ...         

In [36]:
id_vars = ["chr", "loc", "left_flank", "repeat", "right_flank"]

old_repeats_long = old_repeats.reset_index().melt(
    id_vars=id_vars,
    var_name="sample_id",
    value_name="wavg",
)

new_repeats_long = new_repeats.reset_index().melt(
    id_vars=id_vars,
    var_name="sample_id",
    value_name="wavg",
)

long_comp = old_repeats_long.merge(
    new_repeats_long,
    on=[*id_vars, "sample_id"],
    how="inner",
    suffixes=("_old", "_new"),
)

long_comp["se"] = np.square(long_comp["wavg_old"] - long_comp["wavg_new"])
long_comp["abs_dev"] = np.abs(long_comp["wavg_old"] - long_comp["wavg_new"])
long_comp["abs_dev_rank"] = long_comp["abs_dev"].rank(ascending=False, method="min")

long_comp

,chr,loc,left_flank,repeat,right_flank,sample_id,wavg_old,wavg_new,se,abs_dev,abs_dev_rank
0,chr1,100077146,AAATC,15[T],CAGAC,CDS-1tmwd7,11.000000,11.000000,0.000000,0.0000,6760.0
1,chr1,100218786,AAAAA,12[T],ACTAA,CDS-1tmwd7,10.000000,10.062500,0.003906,0.0625,1595.0
2,chr1,10072379,ATCGC,10[T],ATTGC,CDS-1tmwd7,8.413793,8.413793,0.000000,0.0000,6760.0
3,chr1,100990636,CTATT,10[A],GATAC,CDS-1tmwd7,9.000000,9.000000,0.000000,0.0000,6760.0
4,chr1,103107961,ATTGC,14[T],CTTGA,CDS-1tmwd7,12.103448,12.103448,0.000000,0.0000,6760.0
...,...,...,...,...,...,...,...,...,...,...,...
45243,chr9,94301443,AATGG,11[A],TGTTG,CDS-0Eax8o,11.019231,11.019231,0.000000,0.0000,6760.0
45244,chr9,94664048,GGAAT,13[A],GCCAG,CDS-0Eax8o,13.000000,13.000000,0.000000,0.0000,6760.0
45245,chr9,95012920,TTTTG,12[T],AACGT,CDS-0Eax8o,12.000000,12.000000,0.000000,0.0000,6760.0
45246,chr9,96645797,TCTCG,14[A],GGAAA,CDS-0Eax8o,14.000000,14.000000,0.000000,0.0000,6760.0


In [37]:
overall_mse = long_comp["se"].mean()
overall_med_abs_dev = long_comp["abs_dev"].median()

print(f"Overall mean squared error: {overall_mse}")
print(f"Overall median absolute deviation: {overall_med_abs_dev}")

Overall mean squared error: 0.007188994356492601
Overall median absolute deviation: 0.0


In [38]:
long_comp.sort_values("abs_dev_rank", ascending=True).head(20)

,chr,loc,left_flank,repeat,right_flank,sample_id,wavg_old,wavg_new,se,abs_dev,abs_dev_rank
7378,chr20,4859251,AAAAT,13[A],GTGTG,CDS-0sdrSB,6.000000,13.000000,49.000000,7.000000,1.0
34350,chr10,7585842,AAACC,11[A],CATTA,CDS-0A4mDu,5.000000,11.000000,36.000000,6.000000,2.0
5445,chr8,132972576,TTTTG,12[T],CCACC,CDS-00Nrci,19.000000,14.333333,21.777778,4.666667,3.0
3098,chr1,65572290,AGTTG,27[T],AAATT,CDS-00Nrci,23.200000,27.500000,18.490000,4.300000,4.0
7392,chr20,59948037,TCTTT,11[A],GAAAA,CDS-0sdrSB,17.833333,13.666667,17.361111,4.166667,5.0
8696,chr1,32680338,TGGTG,11[T],GTTGT,CDS-0bV15m,12.916667,9.545455,11.365071,3.371212,6.0
3357,chr11,85635821,TTTTC,18[T],AGGTT,CDS-00Nrci,20.833333,17.954545,8.287420,2.878788,7.0
3763,chr15,29718124,TGTTT,20[A],GACAA,CDS-00Nrci,17.000000,19.571429,6.612245,2.571429,8.0
5641,chr9,86035671,GATTT,22[A],GCAAA,CDS-00Nrci,25.000000,22.500000,6.250000,2.500000,9.0
3363,chr11,89927655,GAGTC,20[A],GAAAA,CDS-00Nrci,21.400000,23.866667,6.084444,2.466667,10.0


## Per site

In [39]:
by_pos = (
    long_comp.groupby(id_vars)["se"]
    .mean()
    .reset_index()
    .merge(
        long_comp.groupby(id_vars)["abs_dev"].median().reset_index(),
        how="inner",
        on=id_vars,
    )
)

by_pos["se_rank"] = by_pos["se"].rank(ascending=False, method="min")
by_pos["abs_dev_rank"] = by_pos["abs_dev"].rank(ascending=False, method="min")

In [40]:
by_pos.loc[by_pos["se_rank"].le(10) | by_pos["abs_dev_rank"].le(10)].sort_values("se_rank")

,chr,loc,left_flank,repeat,right_flank,se,abs_dev,se_rank,abs_dev_rank
1722,chr20,4859251,AAAAT,13[A],GTGTG,3.067987,0.002097,1.0,47.0
414,chr10,7585842,AAACC,11[A],CATTA,2.573257,0.000000,2.0,65.0
2617,chr8,132972576,TTTTG,12[T],CCACC,1.526229,0.033333,3.0,17.0
1736,chr20,59948037,TCTTT,11[A],GAAAA,1.303658,0.000000,4.0,65.0
270,chr1,65572290,AGTTG,27[T],AAATT,1.232798,0.000000,5.0,65.0
212,chr1,32680338,TGGTG,11[T],GTTGT,0.791121,0.049784,6.0,14.0
747,chr13,18841488,ATCTC,15[A],CCCAA,0.782656,0.000000,7.0,65.0
529,chr11,85635821,TTTTC,18[T],AGGTT,0.518911,0.000000,8.0,65.0
935,chr15,29718124,TGTTT,20[A],GACAA,0.454508,0.000000,9.0,65.0
534,chr11,89916796,TTTTC,20[T],GACTC,0.442841,0.020170,10.0,23.0


## Per sample

In [41]:
by_sample = (
    long_comp.groupby('sample_id')["se"]
    .mean()
    .reset_index()
    .merge(
        long_comp.groupby('sample_id')["abs_dev"].median().reset_index(),
        how="inner",
        on='sample_id',
    )
)

by_sample["se_rank"] = by_sample["se"].rank(ascending=False, method="min")
by_sample["abs_dev_rank"] = by_sample["abs_dev"].rank(ascending=False, method="min")

In [42]:
by_sample.loc[by_sample["se_rank"].le(10) | by_sample["abs_dev_rank"].le(10)].sort_values("se_rank")

,sample_id,se,abs_dev,se_rank,abs_dev_rank
11,CDS-0sdrSB,0.040119,0.014049,1.0,1.0
0,CDS-00Nrci,0.035312,0.000000,2.0,3.0
4,CDS-0A4mDu,0.014194,0.000000,3.0,3.0
12,CDS-1tmwd7,0.009228,0.001732,4.0,2.0
8,CDS-0bV15m,0.004534,0.000000,5.0,3.0
15,CDS-xvtYY4,0.003501,0.000000,6.0,3.0
9,CDS-0e3PRe,0.002624,0.000000,7.0,3.0
1,CDS-01m27W,0.001196,0.000000,8.0,3.0
3,CDS-099jzP,0.001119,0.000000,9.0,3.0
13,CDS-BEbnKg,0.001076,0.000000,10.0,3.0
